# Visualizing Price Movement

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import sklearn as sk
import statsmodels.api as sm

In [2]:
#path to the data folder, edit if necesssary
path = "./data/"

#filenames
file_power = "competitor_market_prices_power.csv"
file_gas = "competitor_market_prices_gas.csv"
master_power = "master_competitor_market_prices_power.csv"
master_gas = "master_competitor_market_prices_gas.csv"

#Price Filenames
power_prices = 'power_prices'
gas_prices = 'gas_prices'

In [13]:
#Reading in gas and power prices
df_power_prices = pd.read_csv(path + power_prices, index_col = 'Dates')
df_gas_prices = pd.read_csv(path + gas_prices, index_col = 'date')

df_power_prices

,spot_peak,spot_base,y_ahead_peak,2y_ahead_peak,3y_ahead_peak,y_ahead_base,2y_ahead_base,3y_ahead_base,d_ahead_peak,d_ahead_base,m_ahead_base,6m_ahead_base,m_ahead_peak,6m_ahead_peak
Dates,,,,,,,,,,,,,,
2000-06-16,20.503333,17.126250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-17,15.180000,13.297083,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-18,12.578333,11.401667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-19,21.919167,17.666250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000-06-20,22.856667,18.363750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-10-10,186.518333,162.090000,581.64,353.67,254.50,427.00,270.00,195.00,186.52,162.09,344.89,389.71,470.80,492.09
2022-10-11,291.780833,259.873750,585.40,355.50,253.50,435.64,270.00,194.92,291.78,259.87,345.31,398.27,469.67,492.92
2022-10-12,299.796667,282.535833,590.00,355.46,253.61,434.00,268.00,193.88,299.80,282.54,341.97,389.53,454.50,485.41


In [7]:
#Reading in master data
df_power = pd.read_csv(path + master_power)
df_power.set_index(pd.to_datetime(df_power['date_valid_from']), inplace = True)
df_gas = pd.read_csv(path + master_gas)
df_gas.set_index(pd.to_datetime(df_gas['date_valid_from']), inplace = True)

In [222]:
df_gas_prices.index = pd.to_datetime(df_gas_prices.index)
df_power_prices.index = pd.to_datetime(df_power_prices.index)

In [316]:
#Segmenting data according to strat_index
df_power_0 = df_power_test.loc[df_power_test.strat_index == 0].copy()
df_power_1 = df_power_test.loc[df_power_test.strat_index == 1].copy()
df_power_2 = df_power_test.loc[df_power_test.strat_index == 2].copy()
df_power_3 = df_power_test.loc[df_power_test.strat_index == 3].copy()
df_power_4 = df_power_test.loc[df_power_test.strat_index == 4].copy()
df_power_5 = df_power_test.loc[df_power_test.strat_index == 5].copy()

In [209]:
#Defining functions that subset our data for a certain consumption range, postcode and supplier name and prepares it for analysis.
#Average household prices are exploded across time period and grouped by validity dates.

def queryRangePostCode(consumption_range_kwh = 2000, post_code = 81737) -> pd.DataFrame:
    res = df_power.loc[(df_power.consumption_range_kwh == consumption_range_kwh) & (df_power.post_code == post_code)].copy()
    
    #Generating date indexed data
    res["valid_range"] = res.apply(lambda x: pd.date_range(x["date_valid_from"], x["date_valid_to"]), axis=1)
    res = res.explode("valid_range").copy()

    #Aggregating for mean household prices per day
    res = res.groupby('valid_range').agg({'price_year_eur':'mean'})
    return res

def queryRangeSupplier(consumption_range_kwh = 2000, supplier = 'Sarma') -> pd.DataFrame:
    res = df_power.loc[(df_power.consumption_range_kwh == consumption_range_kwh) & (df_power.supplier == supplier)].copy()

    #Generating date indexed data
    res["valid_range"] = res.apply(lambda x: pd.date_range(x["date_valid_from"], x["date_valid_to"]), axis=1)
    res = res.explode("valid_range").copy()

    #Aggregating for mean household prices per day
    res = res.groupby('valid_range').agg({'price_year_eur':'mean'})
    return res

def queryPostCodeSupplier(post_code = 81737, supplier = 'Sarma') -> pd.DataFrame:
    res = df_power.loc[(df_power.post_code == post_code) & (df_power.supplier == supplier)].copy()

    #Generating date indexed data
    res["valid_range"] = res.apply(lambda x: pd.date_range(x["date_valid_from"], x["date_valid_to"]), axis=1)
    res = res.explode("valid_range").copy()

    #Aggregating for mean household prices per day
    res = res.groupby('valid_range').agg({'price_year_eur':'mean'})
    return res

def queryPower(consumption_range_kwh = 2000, supplier = 'Sarma', post_code = 81737) -> pd.DataFrame:
    res = df_power.loc[(df_power.consumption_range_kwh == consumption_range_kwh) & (df_power.supplier == supplier) & (df_power.post_code == post_code)].copy()

    #Generating date indexed data
    res["valid_range"] = res.apply(lambda x: pd.date_range(x["date_valid_from"], x["date_valid_to"]), axis=1)
    res = res.explode("valid_range").copy()

    #Aggregating for mean household prices per day
    res = res.groupby('valid_range').agg({'price_year_eur':'mean'})
    return res

In [238]:
#Defining function for price time series preparation.
#price_instrument -> str specifying desired price instrument
#model_type -> str, either 'ewm' or 'sma'
#days -> int, how many days is the model looking back for sma models, and maps to alpha (by dividing by 1000) for ewm models

def selectPriceTimeSeries(price_instrument = 'y_ahead_base', model_type = 'sma', days = 100) -> pd.DataFrame:
    if model_type == 'sma':
        res = df_power_prices[price_instrument]
        res = pd.DataFrame(res.rolling(days).mean())
    else:
        days /= 1000
        res = pd.DataFrame(df_power_prices[[price_instrument]].ewm(alpha = days, adjust = False).mean())
    return res



#Defining function to get most relevant price instrument timeseries combo as measured by linear regression R-squared
#price_instruments -> list of power price instruments to try
#household data -> daily aggregated mean household data to regress upon

from sklearn.linear_model import LinearRegression

def getMostRelevantInstrument(household_data):

    #Model output which will be printed
    max_R2 = 0
    best_model_type = ''
    best_model_period = 0
    best_model_instrument = ''
    price_instruments = ['spot_base', 'y_ahead_base', '3y_ahead_base', 'm_ahead_base', '6m_ahead_base']

    #min_household_date = min(pd.to_datetime(household_data.index))
    #max_household_date = max(pd.to_datetime(household_data.index))

    household_data.set_index(pd.to_datetime(household_data.index))
    models = ['sma', 'ewm']
    times = [100, 200, 300]

    for instrument in price_instruments:
        for model in models:
            for time in times:
                timeseries = selectPriceTimeSeries(instrument, model, time)
                temp = household_data.join(timeseries)

                X = temp[[instrument]]
                Y = np.array(temp.price_year_eur)

                lm = LinearRegression()
                reg = lm.fit(X,Y)
                score = reg.score(X,Y)

                if score > max_R2:
                    max_R2 = score
                    best_model_type = model
                    best_model_instrument = instrument
                    best_model_period = time
    
    print('Most powerful model: ', best_model_type)
    print('Most powerful instrument: ', best_model_instrument)
    print('Used time period: ', best_model_period)
    print('Model R2: ', max_R2)


getMostRelevantInstrument(queryPostCodeSupplier(24109, 'Bergwind')) 





Most powerful model:  sma
Most powerful instrument:  spot_base
Used time period:  100
Model R2:  0.6949688420813755


In [232]:
df_power.post_code.value_counts()

21423    570514
24109    568701
48161    324642
85055    309299
72108    303777
          ...  
22159    279403
48429    278245
40625    277993
67663    276536
90763    274762
Name: post_code, Length: 100, dtype: int64

In [237]:
df_power.supplier.value_counts()

Sarma        1606313
Bergwind     1130787
Caurus       1108600
Pulenat       941280
Belat         898948
              ...   
Br               472
Meltemi          452
Sharki           256
Ora              122
Jetstream         37
Name: supplier, Length: 135, dtype: int64